Below is the code for denoising. I have written a denoising function and below it is the code to run all the files.
They are not yet getting saved.

To run all the files. change the base directory and also change the task_types and number variable back to the commented out section.

In [ ]:
import h5py
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from meegkit import star
from meegkit.utils import demean, normcol

def get_dataset_name(file_path):
    file_name = os.path.basename(file_path)  # Extract the file name
    temp = file_name.split('_')[:-1]
    dataset_name = "_".join(temp)
    return dataset_name

def denoise(matrix):
    """Takes a matrix and applies denoising to it using the STAR method.
    :param: matrix: inputs a matrix.
    :return y: the denoised data of the matrix
            w: I have no idea
            denoised_matrix:  the denoised matrix retransposed to fit the original shape
    """
    transposed_matrix = np.transpose(matrix)
    y, w, _ = star.star(transposed_matrix, 2)
    denoised_matrix = np.transpose(transposed_matrix)
    return y, w, denoised_matrix

#Note that you need to change the base directory
base_directory = os.path.join(os.path.expanduser('~'), 'Downloads', 'FinalProjectdata', 'FinalProjectdata', 'Intra', 'train')
task_types = ['rest'] #['rest', 'task_motor', 'task_story_math', 'task_working_memory']

scaler = MinMaxScaler(feature_range=(0, 1), copy=True, clip=False)

# Loop over task types
for task_type in task_types:
    for number in ['1']: #['1', '2', '3', '4', '5', '6', '7', '8']
        # Construct the file path using os.path.join
        file_path = os.path.join(base_directory, f"{task_type}_105923_{number}.h5")
        
        # Check if the file exists
        if os.path.exists(file_path):
            # Process the file
            with h5py.File(file_path, 'r') as f:
                dataset_name = get_dataset_name(file_path)

                # Check if the dataset exists
                if dataset_name in f:
                    matrix = f[dataset_name][()]
                    # Reshape the matrix to a 1D array if needed

                    # Apply MinMaxScaler
                    scaled_data = scaler.fit_transform(matrix)

                    # Apply denoising
                    y, w, retransposed_matrix = denoise(matrix)

                    # Print the first few instances of the scaled data
                    print("First few instances of the scaled data:")
                    print(scaled_data[:5, :5])
                    print("First few instances of the retransposed data:")
                    print(retransposed_matrix[:5, :5])
                    print()
                else:
                    print(f"Dataset '{dataset_name}' not found in the file: {file_path}")
        else:
            print(f"File not found: {file_path}")

Makes the plots

In [ ]:
import matplotlib.pyplot as plt

#Generates the plot based on the data
f, (ax1, ax2) = plt.subplots(2, 1)
ax1.plot(retransposed_matrix, lw=.5)
ax1.set_title(f"Signal + Artifacts (not scaled)")
ax2.plot(y, lw=.5)
ax2.set_title("Denoised")
# ax3.plot(demean(y) - x0, lw=.5) #I have removed it for now, we might add it back x0 being scaled data transposed
# ax3.set_title("Residual")
f.set_tight_layout(True)
plt.show()

# The example code #

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from meegkit import star
from meegkit.utils import demean, normcol

rng = np.random.default_rng(9)


# Create simulated data
nchans = 10
n_samples = 1000
f = 2
target = np.sin(np.arange(n_samples) / n_samples * 2 * np.pi * f)
target = target[:, np.newaxis]
noise = rng.standard_normal((n_samples, nchans - 3))

# Create artifact signal
SNR = np.sqrt(1)
x0 = normcol(np.dot(noise, rng.standard_normal((noise.shape[1], nchans)))) + \
    SNR * target * rng.standard_normal((1, nchans))
x0 = demean(x0)
artifact = np.zeros(x0.shape)
for k in np.arange(nchans):
    artifact[k * 100 + np.arange(20), k] = 1
x = x0 + 10 * artifact

print(x.shape)

y, w, _ = star.star(x, 2)

In [ ]:
import matplotlib.pyplot as plt

f, (ax1, ax2, ax3) = plt.subplots(3, 1)
ax1.plot(x, lw=.5)
ax1.set_title(f"Signal + Artifacts")
ax2.plot(y, lw=.5)
ax2.set_title("Denoised")
ax3.plot(demean(y) - x0, lw=.5)
ax3.set_title("Residual")
f.set_tight_layout(True)
plt.show()